Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [12]:
CUSTOM_MODEL_NAME = 'hand_sign_recognition_model'

WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images' # Change this for different image directory
TRAIN_PATH = IMAGE_PATH + '/train'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/' + CUSTOM_MODEL_NAME + '/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/' + CUSTOM_MODEL_NAME + '/checkpoints'
import sys
sys.executable

'E:\\Anaconda\\envs\\tf-gpu\\python.exe'

# 1. Create Label Map, Update Num Classes in Config and Empty Previous Model's Checkpoints

In [13]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

labels = []
id_count = 1
for subdir, dirs, files in os.walk(TRAIN_PATH):
  for dir_name in dirs:
    labels.append({'name':dir_name, 'id':id_count})
    id_count+=1      

print(labels)

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Change num_classes in config
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                           
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)  

pipeline_config.model.ssd.num_classes = len(labels)

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                           
    f.write(config_text) 

# Empties old checkpoints, necessary to train a new different model
import os
import shutil
shutil.rmtree(CHECKPOINT_PATH)
os.makedirs(CHECKPOINT_PATH)

train_steps = 5000

print("""\npython {}/research/object_detection/model_main_tf2.py --model_dir={} --pipeline_config_path={}/{}/pipeline.config --num_train_steps={}""".format(APIMODEL_PATH, CHECKPOINT_PATH,MODEL_PATH,CUSTOM_MODEL_NAME, train_steps))

[{'name': 'chair', 'id': 1}, {'name': 'drink', 'id': 2}, {'name': 'go', 'id': 3}, {'name': 'now', 'id': 4}, {'name': 'who', 'id': 5}, {'name': 'yes', 'id': 6}]

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/hand_sign_recognition_model/checkpoints --pipeline_config_path=Tensorflow/workspace/models/hand_sign_recognition_model/pipeline.config --num_train_steps=5000


# 2. Create TF records

In [14]:
!python {SCRIPTS_PATH + '/generate_tfrecords_directories.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecords_directories.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [4]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [8]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [30]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.
'cp' is not recognized as an internal or external command,
operable program or batch file.


# 5. Update Config For Transfer Learning (First time configuring model)

In [17]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [22]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [23]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [11]:
# if in-commented displays configuration file
#config

In [16]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [12]:
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [13]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [48]:
train_steps = 5000
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={} --pipeline_config_path={}/{}/pipeline.config --num_train_steps={}""".format(APIMODEL_PATH, CHECKPOINT_PATH,MODEL_PATH,CUSTOM_MODEL_NAME, train_steps))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet/checkpoints --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


# Training Summary and Evaluation

In [ ]:
# cd to train directory and, use tensorboard --logdir=.
# python model_main_tf2.py --model_dir=Tensorflow\workspace\models\hand_sign_recognition_model\checkpoints --pipeline_config_path=Tensorflow/workspace/models/hand_sign_recognition_model/pipeline.config --checkpoint_dir=Tensorflow\workspace\models\hand_sign_recognition_model\checkpoints
# cd to eval directory, and use tensorboard --logdir=.
# Single values in graphs, higher values better